In [6]:
class binary_tree:
    def __init__(self, rootObj):
        self.key = rootObj
        self.leftChild = None
        self.rightChild = None
        
    def insertLeft(self, newNode):
        if isinstance(newNode, binary_tree):
            t = newNode
        else:
            t = binary_tree(newNode)
            
        if self.leftChild is not None:
            t.leftChild = self.leftChild
        self.leftChild = t
        
    def insertRight(self, newNode):
        if isinstance(newNode, binary_tree):
            t = newNode
        else:
            t = binary_tree(newNode)
            
        if self.rightChild is not None:
            t.rightChild = self.rightChild
        self.rightChild = t
        
    def isLeaf(self):
        return not (self.leftChild or self.rightChild)
    
    def getRightChild(self):
        return self.rightChild
    
    def getLeftChild(self):
        return self.leftChild
    
    def setRootVal(self, obj):
        self.key = obj
        
    def getRootVal(self):
        return self.key
    
    def inOrder(self):
        if self.leftChild:
            self.leftChild.inOrder()
        print(self.key)
        if self.rightChild:
            self.rightChild.inOrder()
        
    def postOrder(self):
        if self.leftChild:
            self.leftChild.postOrder()
        if self.rightChild:
            self.rightChild.postOrder()
        print(self.key)
        
    def preOrder(self):
        print(self.key)
        if self.leftChild:
            self.leftChild.preOrder()
        if self.rightChild:
            self.rightChild.preOrder()
            
    def printexp(self):
        if self.leftChild:
            print('(', end = ' ')
            self.leftChild.printexp()
        print(self.key)
        if self.rightChild:
            self.rightChild.printexp()
            print(')', end = ' ')
            
    def postOrderEval(self):
        opers = {'+':operator.add, '-':operator.sub, '*':operator.mul, '/':operator.truediv}
        res1 = None
        res2 = None
        if self.leftChild:
            res1 = self.leftChild.postOrderEval()
        if self.rightChild:
            res2 = self.rightChild.postOrderEval()
        if res1 and res2:
            return opers[self.key](res1, res2)
        else:
            return self.key

test = binary_tree(1)
test.insertLeft(2)
print(test.getLeftChild().key)
test.insertRight(4)
test.insertRight(5)
print('-----')
test.inOrder()

2
-----
2
1
5
4


#### 1. Extend the buildParseTree function to handle mathematical expressions that do not have spaces between every character.


In [10]:
from pythonds.basic import Stack
from pythonds.trees import BinaryTree

def buildParseTree(fpexp):
    fplist = []
    num = ''
    i = 0
    while i < len(fpexp):
        if fpexp[i] in ['(', ')', '+', '-', '*', '/']:
            fplist.append(fpexp[i])
            i += 1
        else:
            num += fpexp[i] 
            i += 1
            while fpexp[i] not in ['(', ')', '+', '-', '*', '/']:
                num += fpexp[i]
                i += 1
            fplist.append(num)
            num = ''   
            
    print(fplist)
    pStack = Stack()
    eTree = BinaryTree('')
    pStack.push(eTree)
    currentTree = eTree

    for i in fplist:
        if i == '(':
            currentTree.insertLeft('')
            pStack.push(currentTree)
            currentTree = currentTree.getLeftChild()

        elif i in ['+', '-', '*', '/']:
            currentTree.setRootVal(i)
            currentTree.insertRight('')
            pStack.push(currentTree)
            currentTree = currentTree.getRightChild()

        elif i == ')':
            currentTree = pStack.pop()

        elif i not in ['+', '-', '*', '/', ')']:
            try:
                currentTree.setRootVal(int(i))
                parent = pStack.pop()
                currentTree = parent

            except ValueError:
                raise ValueError("token '{}' is not a valid integer".format(i))

    return eTree

pt = buildParseTree("((10+5)*3)")
pt.postorder()  #defined and explained in the next section


['(', '(', '10', '+', '5', ')', '*', '3', ')']
10
5
+
3
*


#### 2. Modify the buildParseTree and evaluate functions to handle boolean statements (and, or, and not). Remember that “not” is a unary operator, so this will complicate your code somewhat.


In [33]:
from pythonds.basic import Stack
from pythonds.trees import BinaryTree
import operator

def buildParseTree(fpexp):
    fplist = fpexp.split()
    pStack = Stack()
    eTree = BinaryTree('')
    pStack.push(eTree)
    currentTree = eTree
    
    for i in fplist:
        if i == '(':
            currentTree.insertLeft('')
            pStack.push(currentTree)
            currentTree = currentTree.getLeftChild()
        elif i in ['+', '-', '*', '/', 'and', 'or']:
            currentTree.setRootVal(i)
            currentTree.insertRight('')
            pStack.push(currentTree)
            currentTree = currentTree.getRightChild()
        elif i == ')':
            currentTree = pStack.pop()
        elif i == 'not':
            currentTree.setRootVal(i)
            currentTree.insertLeft('')
            pStack.push(currentTree)
            currentTree = currentTree.getLeftChild()
        elif i not in ['+', '-', '*', '/', 'and', 'or', 'not', '(', ')']:
            try:
                currentTree.setRootVal(int(i))
                parent = pStack.pop()
                currentTree = parent
            except ValueError:
                raise ValueError("token '{}' is not a valid integer".format(i))
                
    return eTree

def evaluate(parseTree):
    opers = {'+':operator.add, '-':operator.sub, '*':operator.mul, '/':operator.truediv, 
             'and':operator.and_, 'or':operator.or_, 'not':operator.not_}
    leftC = parseTree.getLeftChild()
    rightC = parseTree.getRightChild()
    if leftC and rightC:
        fn = opers[parseTree.getRootVal()]
        return fn(evaluate(leftC), evaluate(rightC))
    else:
        return parseTree.getRootVal()


pt = buildParseTree("( ( ( 2 + 3 ) * 3 ) or ( not ( 2 and 11 ) ) )")
pt.postorder()
#print(evaluate(pt))

2
3
+
3
*
2
11
and
not

or


In [31]:
_

''

#### 3. Using the findSuccessor method, write a non-recursive inorder traversal for a binary search tree.


Modify the code for a binary search tree to make it threaded. Write a non-recursive inorder traversal method for the threaded binary search tree. A threaded binary tree maintains a reference from each node to its successor.
Modify our implementation of the binary search tree so that it handles duplicate keys properly. That is, if a key is already in the tree then the new payload should replace the old rather than add another node with the same key.
Create a binary heap with a limited heap size. In other words, the heap only keeps track of the n most important items. If the heap grows in size to more than n items the least important item is dropped.
Clean up the printexp function so that it does not include an ‘extra’ set of parentheses around each number.
Using the buildHeap method, write a sorting function that can sort a list in O(nlogn) time.
Write a function that takes a parse tree for a mathematical expression and calculates the derivative of the expression with respect to some variable.
Implement a binary heap as a max heap.
Using the BinaryHeap class, implement a new class called PriorityQueue. Your PriorityQueue class should implement the constructor, plus the enqueue and dequeue methods.